## intro

A notebook is a good way to document code and share your thoughts with others (and with yourself in the future). Interestingly, *github displays notebooks beautifully*. By checking in this file to a public repository, github serves my blog. Since it is tracked in git, as I publish updates to it, all the history is maintained - pretty cool.

## random thoughts 

There is a division operator - ÷ - that is the same as "/". Julia is a new language that has embraced unicode. This is a huge difference from MATLAB and Python. I'm not comfortable with it yet. To enter the division symbol in this Julia notebook, you can type `\div<tab>`. As a "vi" user, I am not able to enter these in my favorite go-to editor, besides to copy and paste from another location - TODO for me is to find how to do it in "vi". I also use "eclipse" and I haven't found out how to cleanly handle unicode there.

In [1]:
7÷2

3

Julia supports "implicit multiplication". So you can write code that is more math like: instead of 3*x you can just write 3x.

In [2]:
x = 5

5

In [3]:
3x

15

In [4]:
3 * x   # this works with spaces around the operator

15

In [5]:
3 x  # this doesn't - implicit multiplication requires NO SPACE

LoadError: [91msyntax: extra token "x" after end of expression[39m